In [ ]:
###############################################################################
# Example MILP for Battery Energy Storage Dispatch in ERCOT
# Using Pyomo (https://pyomo.readthedocs.io/) and a generic solver interface.
###############################################################################

import pyomo.environ as pyo

def create_ercot_storage_model(time_horizon=24, interval_minutes=15):
    """
    Create a MILP model for an energy storage system (ESS) providing stacked
    services in ERCOT. Fill in your own data for LMPs, ancillary services, etc.
    """

    # Number of intervals in one day (if 15-min intervals, 24*4 = 96 intervals)
    steps_per_hour = 60 // interval_minutes
    num_periods = time_horizon * steps_per_hour

    # -------------------------
    # 1) Create the model
    # -------------------------
    model = pyo.ConcreteModel("ERCOT_Battery_Stacking")

    # -------------------------
    # 2) Sets and Parameters
    # -------------------------
    # Set of time periods (0..num_periods-1)
    model.T = pyo.RangeSet(0, num_periods - 1)

    # Parameters you must provide (from your ERCOT data/forecasts):
    # These can be read from CSV or external data frames, e.g. using pandas.
    # For demonstration, we assume these are Python lists indexed by t.

    # Example energy market price [$/MWh], length = num_periods
    # (Replace with ERCOT LMP or your internal price vector)
    model.energy_price = pyo.Param(
        model.T, initialize=lambda m, t: 35.0, mutable=True
    )

    # Regulation Up price [$/MW per interval], length = num_periods
    model.reg_up_price = pyo.Param(
        model.T, initialize=lambda m, t: 10.0, mutable=True
    )

    # Regulation Down price [$/MW per interval], length = num_periods
    model.reg_down_price = pyo.Param(
        model.T, initialize=lambda m, t: 8.0, mutable=True
    )

    # Non-Spin / RRS price [$/MW per interval], length = num_periods
    model.nonspin_price = pyo.Param(
        model.T, initialize=lambda m, t: 5.0, mutable=True
    )

    # Battery parameters (set to your system’s specs):
    # Battery max power in MW
    model.Pmax = pyo.Param(initialize=1.0, mutable=True)  # e.g. 1 MW
    # Battery energy capacity in MWh
    model.Emax = pyo.Param(initialize=2.0, mutable=True)  # e.g. 2 MWh
    # Round-trip efficiency (fraction, e.g. 0.9)
    model.eta = pyo.Param(initialize=0.90, mutable=True)
    # Min/Max State of Charge
    model.SOC_min = pyo.Param(initialize=0.0)
    model.SOC_max = pyo.Param(initialize=1.0)

    # Duration of each interval in hours
    dt = interval_minutes / 60.0
    model.dt = pyo.Param(initialize=dt)

    # You can also store a big-M parameter for certain constraints if needed
    # (not always mandatory, but often used for binary commit variables, etc.)
    model.BigM = pyo.Param(initialize=model.Pmax.value)

    # -------------------------
    # 3) Decision Variables
    # -------------------------
    # Battery SoC in fraction of Emax (0..1)
    model.soc = pyo.Var(model.T, bounds=(0, 1), initialize=0.5)

    # Power charged (MW) at each time period
    model.charge_power = pyo.Var(model.T, bounds=(0, None), initialize=0.0)

    # Power discharged (MW) at each time period
    model.discharge_power = pyo.Var(model.T, bounds=(0, None), initialize=0.0)

    # Regulation Up capacity offered (MW)
    model.reg_up = pyo.Var(model.T, bounds=(0, None), initialize=0.0)

    # Regulation Down capacity offered (MW)
    model.reg_down = pyo.Var(model.T, bounds=(0, None), initialize=0.0)

    # Non-Spin (RRS) capacity offered (MW)
    model.nonspin = pyo.Var(model.T, bounds=(0, None), initialize=0.0)

    # In real market designs, you may need additional variables to track
    # exactly how much energy is used for each service, or you must ensure
    # you’re not “double-counting” capacity. We keep it simple here.

    # -------------------------
    # 4) Constraints
    # -------------------------

    # (a) The battery cannot charge and discharge above Pmax
    #     Also ensure that total offered capacity in any interval
    #     cannot exceed physical rating, e.g. reg_up + discharge <= Pmax, etc.
    def power_limit_rule(m, t):
        return (
            m.charge_power[t] + m.discharge_power[t]
            + m.reg_up[t] + m.reg_down[t] + m.nonspin[t]
            <= m.Pmax
        )
    model.PowerLimit = pyo.Constraint(model.T, rule=power_limit_rule)

    # (b) State-of-Charge dynamics
    #    soc[t+1] = soc[t] + (charge_power[t]*eta - discharge_power[t]/eta)*dt / Emax
    def soc_evolution_rule(m, t):
        if t == num_periods - 1:
            # For a 24hr horizon, you might want to enforce final SoC = initial
            # or let it free. We keep the final unconstrained except for min/max.
            return pyo.Constraint.Skip
        return (
            m.soc[t + 1]
            == m.soc[t]
            + (
                m.charge_power[t] * m.eta
                - m.discharge_power[t] / m.eta
            )
            * m.dt
            / (m.Emax)
        )
    
    model.SOC_Evolution = pyo.Constraint(model.T, rule=soc_evolution_rule)

    # (c) SoC bounds: This is already embedded in variable bounds (0..1),
    #     but if you prefer a direct MWh approach, do it here.
    #     e.g. 0 <= SoC[t] <= Emax. We skip that since we used fraction.

    # (d) Possibly prevent simultaneous charge & discharge:
    #     (You can do this with a binary or the simpler approach below)
    def no_simultaneous_charge_discharge_rule(m, t):
        return m.charge_power[t] * m.discharge_power[t] == 0
    model.NoSimultaneous = pyo.Constraint(model.T, rule=no_simultaneous_charge_discharge_rule)

    # (e) If you need to ensure that the battery’s real-time net output
    #     plus reg_up, reg_down, etc. is physically feasible, you might
    #     add constraints reflecting how much headroom is left for up/down.
    #     Typically: discharge_power[t] + reg_up[t] <= Pmax, and so on.

    # Example:
    def reg_up_feasibility_rule(m, t):
        # reg_up requires the battery to have *discharge* headroom
        # (i.e. SoC>0). Must ensure the system can ramp up from 0
        # to reg_up[t] if it’s not currently discharging. This is simplified.
        return m.discharge_power[t] + m.reg_up[t] <= m.Pmax
    model.RegUpFeasibility = pyo.Constraint(model.T, rule=reg_up_feasibility_rule)

    def reg_down_feasibility_rule(m, t):
        # reg_down requires *charging* headroom
        return m.charge_power[t] + m.reg_down[t] <= m.Pmax
    model.RegDownFeasibility = pyo.Constraint(model.T, rule=reg_down_feasibility_rule)

    # Non-spin might require the battery to be idle or partially idle
    # so it can provide that capacity upon request, etc.
    # Adjust as needed for actual ERCOT rules.

    # -------------------------
    # 5) Objective Function
    # -------------------------
    # Maximize net revenue from:
    # (1) Energy arbitrage: (discharge_power * LMP - charge_power * LMP)
    # (2) Reg Up capacity, Reg Down capacity, Non-Spin capacity
    #
    # If you have mileage-based payments (like pay-for-performance),
    # you can integrate that as well. For simplicity, assume capacity-only.

    def revenue_rule(m):
        # Summation over T of [ Energy revenue + ancillary revenue ]
        rev = 0
        for t in m.T:
            # energy sold
            rev += (m.discharge_power[t] * m.energy_price[t] * m.dt)
            # energy purchased is a negative revenue
            rev -= (m.charge_power[t] * m.energy_price[t] * m.dt)

            # Regulation Up capacity payment
            rev += m.reg_up[t] * m.reg_up_price[t] * m.dt
            # Regulation Down capacity payment
            rev += m.reg_down[t] * m.reg_down_price[t] * m.dt
            # Non-spin capacity payment
            rev += m.nonspin[t] * m.nonspin_price[t] * m.dt

        return rev

    model.Obj = pyo.Objective(rule=revenue_rule, sense=pyo.maximize)

    return model


    


In [21]:
import pandas as pd
import numpy as np

ancilary_service = pd.read_csv('/Users/leroy/Documents/GitHub/Electricity_trading/Battery_opt/Data/ancillary_service_2024.csv')

ancilary_service.columns

ancilary_service.iloc[:,3:].mean()

REGDN     3.439229
REGUP     6.007119
RRS       5.674302
NSPIN     6.222699
ECRS      8.231750
dtype: float64

In [22]:
DAM_price = pd.read_csv('/Users/leroy/Documents/GitHub/Electricity_trading/DART/Data/2023_24_ERCOT_forecast_actual_data_v3.csv')

DAM_price.DA_Price_North.mean()

49.389340561677926

In [8]:
ancilary_service_one = ancilary_service[ancilary_service['Delivery Date'] == '01/01/2024'].copy()

print(ancilary_service_one.head())

DAM_one = DAM_price[DAM_price.marketday == '1/1/2024'].copy()

print(DAM_one.head())

  Delivery Date Hour Ending Repeated Hour Flag  REGDN  REGUP    RRS  NSPIN  \
0    01/01/2024       01:00                  N   1.51    1.49  1.00   0.94   
1    01/01/2024       02:00                  N   1.25    1.47  1.00   0.94   
2    01/01/2024       03:00                  N   1.25    1.80  1.00   1.23   
3    01/01/2024       04:00                  N   1.25    1.60  1.00   1.23   
4    01/01/2024       05:00                  N   1.25    2.00  1.01   1.23   

   ECRS  
0  0.10  
1  0.11  
2  0.12  
3  0.13  
4  0.14  
     marketday  hourending  SP_Price_Houston  SP_Price_North  SP_Price_Panh  \
7871  1/1/2024           1           13.8800         14.8700        15.2750   
7872  1/1/2024           2           15.9775         16.0900        16.4050   
7873  1/1/2024           3           16.9950         16.9950        16.9950   
7874  1/1/2024           4           17.9525         17.9525        17.9525   
7875  1/1/2024           5           20.6575         20.6575        20.6575 

In [ ]:
# Create the model (one-day horizon, 15-min intervals)
model = create_ercot_storage_model(time_horizon=24, interval_minutes=60)

# Here you would load actual data into model.energy_price[t],
# model.reg_up_price[t], etc., from your CSV or database:
    #
    # for t in model.T:
    #     model.energy_price[t] = my_energy_prices[t]
    #     model.reg_up_price[t] = my_reg_up_prices[t]
    #     ...
    #
    # Also set battery size or other parameters if needed.

    # Solve using (for example) CBC or GLPK or Gurobi:
solver = pyo.SolverFactory("cbc")  # or "gurobi", "glpk", etc.
result = solver.solve(model, tee=True)

    # Check solver status
print("Solver status:", result.solver.status)
print("Solver termination condition:", result.solver.termination_condition)

    # If optimal, retrieve results:
for t in model.T:
    print(
            f"t={t}, "
            f"Charge={pyo.value(model.charge_power[t]):.2f} MW, "
            f"Discharge={pyo.value(model.discharge_power[t]):.2f} MW, "
            f"SoC={pyo.value(model.soc[t]):.2f}"
        )

    # The final objective value (total daily revenue):
total_revenue = pyo.value(model.Obj)
print("Total revenue =", total_revenue)